In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str)
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str)

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras = pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id')

In [10]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')
not check1.empty and print('Hay proveedores no calificados')
check1


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora


# Pruebas

In [11]:
compras_oficiales = compras.loc[compras['oficial'] == True]

In [12]:
ventas_año['costo_producto'] = pd.NA
ventas_año['costo_order_date'] = pd.NaT
ventas_año['costo_order_line_id'] = pd.NA

for i in range(len(ventas_año)):

    minidf = (compras[
            (compras['order_date'] <= pd.to_datetime(ventas_año['invoice_date'].iloc[i]) + pd.Timedelta(days=15)) 
            & (compras['product_id_pp'] == ventas_año['product_id'].iloc[i])]
            .sort_values('order_date', ascending = False)
            )

    if not minidf.empty:
        ventas_año.iloc[i,30] = minidf['product_price'].iloc[0]
        ventas_año.iloc[i,31] = minidf['order_date'].iloc[0]
        ventas_año.iloc[i,32] = minidf['line_id'].iloc[0]

    else:
        minidf_costo_inicial = productos_sin_compra[productos_sin_compra['product_id'] == ventas_año['product_id'].iloc[i]]

        if not minidf_costo_inicial.empty:
            ventas_año.iloc[i,30] = minidf_costo_inicial['costo_producto'].iloc[0]
            ventas_año.iloc[i,31] = minidf_costo_inicial['costo_sae_date'].iloc[0]

In [14]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]

In [16]:
# ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id').to_excel('wep.xlsx')
ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id')

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,price_unit,discount,price_subtotal,costo_producto,costo_order_date,costo_order_line_id,costo_date_dif,costo_subtotal,utilidad_subtotal,utilidad_%
8149,12947,F2-CC/2024/01676,2024-01-25,posted,PdV SJC/1703,PdV,3037,out_invoice,<NA>,<NA>,...,11.30,0.00,22.60,NaN,2017-08-11,<NA>,2358,NaN,NaN,NaN
10166,16436,F1-CC/2024/01562,2024-01-31,posted,PdV CSL/1559,PdV,3879,out_invoice,<NA>,<NA>,...,1.00,0.10,0.00,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN
10641,17232,F1-CC/2024/01646,2024-02-01,posted,PdV CSL/1620,PdV,4019,out_invoice,<NA>,<NA>,...,1.00,0.00,"4,080.17",<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN
13243,21450,RF1-VS/2024/00002,2024-02-08,posted,<NA>,Contabilidad,<NA>,out_refund,<NA>,<NA>,...,130.74,0.00,-130.74,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN
16962,27768,F1-CC/2024/02560,2024-02-20,posted,PdV CSL/2489,PdV,6255,out_invoice,<NA>,<NA>,...,0.00,0.00,0.00,NaN,2022-10-10,<NA>,498,NaN,NaN,NaN
18271,29861,F1-CC/2024/02770,2024-02-22,posted,PdV CSL/2700,PdV,6726,out_invoice,42460,<NA>,...,843.46,0.00,843.46,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN
56609,97874,F2-CC/2024/11004,2024-06-07,posted,PdV SJC/11194,PdV,20300,out_invoice,<NA>,<NA>,...,95.24,0.00,95.24,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaN
